In [18]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By

import time

In [29]:
def load_soup_object(html_file_name):
    page=requests.get(html_file_name)
    soup=BeautifulSoup(page.content,'html.parser')
    return soup

In [154]:
def write_list_to_file(file_name,list_name):
    with open(file_name, 'w') as f:
        for li in list_name:
            f.write("%s\n" % li)

In [41]:
#scapping articles'urls from the website
#scrapping from multiple pages with 'loas more button' javascript link
#used stackoverflow - https://stackoverflow.com/questions/68792444/how-to-scrape-website-if-it-has-load-more-button-to-load-more-content-on-the-pag
def israel365news_crawler(webdriver_path,max_links):
    driver=webdriver.Chrome(executable_path=webdriver_path)
    soup=load_soup_object('https://www.israel365news.com/israel-news/')

    try:
        driver.maximize_window() #ensures that the window occupies the entire screen.
        driver.implicitly_wait(10) #wait for up to 10 seconds for the element to appear before throwing an exception
        driver.get("https://www.israel365news.com/israel-news/")

        urls=[]

        while max_links>=0:
            if max_links%100==0: print(f'{max_links} urls until finish')
            #articles = driver.find_elements_by_css_selector("h1.elementor-heading-title.elementor-size-default")
            #articles=driver.find_elements("xpath", "//h1[@class='elementor-heading-title elementor-size-default']")
            articles=soup('h1',attrs={"class":"elementor-heading-title elementor-size-default"})

            for art in articles:
                #url=art.find_element_by_tag_name("a").get_attribute("href")
                #url = art.find_element("xpath", "./a").get_attribute("href")
                #urls.append(url)
                #print(f'url appended: {url}')
                a=art.find('a')
                try:
                    if 'href' in a.attrs:
                        url=a.get('href')
                        urls.append(url)
                        #print(f'url appended: {url}')
                        max_links-=1
                except:
                    continue

            #loadmore = driver.find_element_by_css_selector("a.elementor-button-link.elementor-button")
            loadmore=driver.find_element("xpath", "//a[contains(@class, 'elementor-button-link') and contains(@class, 'elementor-button')]")

            if loadmore.is_displayed():
                driver.execute_script("arguments[0].click();", loadmore)
                time.sleep(5)
            else:
                print("no more articles to load")
                break

    except StaleElementReferenceException:
        pass

    finally:
        driver.quit()
    return urls


In [42]:
webdriver_path=r"C:\Users\hadar\Documents\chromedriver_win32\chromedriver.exe"
urls_list=israel365news_crawler(webdriver_path,10000)


C:\Users\hadar\AppData\Local\Temp\ipykernel_6828\2713619630.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=webdriver_path)


10000 urls until finish
9900 urls until finish
9800 urls until finish
9700 urls until finish
9600 urls until finish
9500 urls until finish
9400 urls until finish
9300 urls until finish
9200 urls until finish
9100 urls until finish
9000 urls until finish
8900 urls until finish
8800 urls until finish
8700 urls until finish
8600 urls until finish
8500 urls until finish
8400 urls until finish
8300 urls until finish
8200 urls until finish
8100 urls until finish
8000 urls until finish
7900 urls until finish
7800 urls until finish
7700 urls until finish
7600 urls until finish
7500 urls until finish
7400 urls until finish
7300 urls until finish
7200 urls until finish
7100 urls until finish
7000 urls until finish
6900 urls until finish
6800 urls until finish
6700 urls until finish
6600 urls until finish
6500 urls until finish
6400 urls until finish
6300 urls until finish
6200 urls until finish
6100 urls until finish
6000 urls until finish
5900 urls until finish
5800 urls until finish
5700 urls 

OSError: [Errno 22] Invalid argument: 'https://www.israel365news.com/israel-news/'

In [45]:
write_list_to_file('israel365news_urls.txt',urls_list)

In [111]:
#crawl through all articles from Israel365News we've collected
def load_articles_to_df_israel365news(urls_list):
    headline=[]
    writers=[]
    date=[]
    genre=[]
    content=[]
    i=0
    
    for url in urls_list:
        soup=load_soup_object(url)
        headline.append(soup.find("h1",attrs={"class":"elementor-heading-title elementor-size-default"}).get_text().strip())
        writers.append(soup.find("h4",attrs={"class":"elementor-author-box__name"}).get_text().strip())
        date.append(soup.find("div",attrs={"class":"elementor-element elementor-element-8f04d6d elementor-widget__width-auto elementor-widget elementor-widget-heading"}).find("h2").get_text().strip())
        genre.append(", ".join(item.get_text().strip() for item in soup.find("div",attrs={"class":"elementor-element elementor-element-1a40dbf elementor-widget__width-auto text-hover elementor-widget elementor-widget-heading"}).find_all("a")))
        content.append(soup.find("div",attrs={"class":"elementor-element elementor-element-2106a85 elementor-widget elementor-widget-theme-post-content"}).get_text().strip()) 
        i+=1
        if(i%100==0):
            print(f'{i} articles scrapped')

    df=pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [112]:
df=load_articles_to_df_israel365news(urls_list)

100 articles scrapped
200 articles scrapped
300 articles scrapped
400 articles scrapped
500 articles scrapped
600 articles scrapped
700 articles scrapped
800 articles scrapped
900 articles scrapped
1000 articles scrapped
1100 articles scrapped
1200 articles scrapped
1300 articles scrapped
1400 articles scrapped
1500 articles scrapped
1600 articles scrapped
1700 articles scrapped
1800 articles scrapped
1900 articles scrapped
2000 articles scrapped
2100 articles scrapped
2200 articles scrapped
2300 articles scrapped
2400 articles scrapped
2500 articles scrapped
2600 articles scrapped
2700 articles scrapped
2800 articles scrapped
2900 articles scrapped
3000 articles scrapped
3100 articles scrapped
3200 articles scrapped
3300 articles scrapped
3400 articles scrapped
3500 articles scrapped
3600 articles scrapped
3700 articles scrapped
3800 articles scrapped
3900 articles scrapped
4000 articles scrapped
4100 articles scrapped
4200 articles scrapped
4300 articles scrapped
4400 articles scrapp

In [113]:
df.to_csv('israel365news.csv', index=False)

In [114]:
df

,headline,writers,date,genre,content
0,Knesset passes 2023-2024 state budget,JNS,"May 24, 2023",Israel News,Israel’s Knesset on Wednesday passed the state...
1,Will flights from Israel to Mecca spark Temple...,Adam Eliyahu Berkowitz,"May 23, 2023",Israel News,Prime Minister Netanyahu was in direct contact...
2,Rabbi Leo Dee weighs $1.3b. suit against Amanpour,JNS,"May 23, 2023",Israel News,Rabbi Leo Dee is considering suing CNN chief i...
3,Israeli Cabinet boosts funds for Kotel infrast...,JNS,"May 22, 2023",Israel News,The Israeli Cabinet on Sunday approved funding...
4,Ben-Gvir visits Temple Mount: ‘We are the owne...,JNS,"May 21, 2023",Israel News,During a visit to the Temple Mount in Jerusale...
...,...,...,...,...,...
10015,Israeli forces kill two terrorists in raid nea...,JNS,"May 10, 2023","Israel News, Judea and Samaria",Israeli forces killed two Palestinian terroris...
10016,IDF eliminates three top Palestinian Islamic J...,JNS,"May 9, 2023","Israel News, Terror Watch",The Israel Defense Forces killed three top Pal...
10017,"After six years, Israel razes illegal PA struc...",JNS,"May 8, 2023",Israel News,Israel’s Civil Administration demolished a Pal...
10018,Jerusalem Day: Mass return to the Temple Mount...,Adam Eliyahu Berkowitz,"May 7, 2023",Israel News,Temple Mount advocacy groups are planning some...


In [169]:
# making crawler for Middle East Monitor website
def load_articles_url_MEM(n_pages):
    soup=load_soup_object(r'https://www.middleeastmonitor.com/category/news/middle-east/')
    articles=soup("li")
    urls=[]
    
    for page in range(n_pages):
        for art in articles:
            a=art.find('a')
            try:
                if 'href' in a.attrs:
                    url=a.get('href')
                    urls.append(url)
            except:
                continue
        next_page=soup.find("a",attrs={"class":"next page-numbers"})["href"]
        soup=load_soup_object(next_page)
        articles=soup("li")
        if(page%50==0): 
            print(f'{page} pages out of 350')
        
    return urls

In [170]:
mem_urls_list=load_articles_url_MEM(350)

0 pages out of 350
50 pages out of 350
100 pages out of 350
150 pages out of 350
200 pages out of 350
250 pages out of 350
300 pages out of 350


In [173]:
#removing links to other sites from list
for url in mem_urls_list:
    if(url.startswith("https://www.middleeastmonitor.com")==False):
        mem_urls_list.remove(url)
write_list_to_file('mem_urls.txt',mem_urls_list)

In [ ]:
#crawl through all articles from Israel365News we've collected
def load_articles_to_df_mem(urls_list):
    headline=[]
    writers=[]#there is no authors declared inside the articles, will be filled with Unknown
    date=[]
    genre=[]
    content=[]
    i=0
    
    for url in urls_list:
        try:
            soup=load_soup_object(url)
            headline.append(soup.find("div",attrs={"id":"post-page-title"}).get_text().strip())
            writers.append("Unknown")
            date.append(soup.find("span",attrs={"class":"post-page-date"}).get_text().strip())
            genre.append(", ".join(item.get_text().strip() for item in soup.find("div",attrs={"class":"post-meta-top"}).find_all("a")))
            content.append((" ".join(item.get_text().strip() for item in soup.find("div",attrs={"id":"post-content"}).find_all("p"))))
            i+=1
            if(i%100==0):
                print(f'{i} articles scrapped')
            if(i>=10000):
                break
        except:
            continue

    df=pd.DataFrame({"headline":headline,"writers":writers,"date":date,"genre":genre,"content":content})
    return df

In [ ]:
df_mem=load_articles_to_df_mem(mem_urls_list)

In [ ]:
df_mem.to_csv('mem.csv', index=False)
df_mem